# First Lab

In [1]:
import os
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

### Creating Dataset Class to Model

In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dogs_path: str, cats_path: str):
        super().__init__()
        self.dogs_path = dogs_path
        self.cats_path = cats_path
        self.dogs_list = sorted(os.listdir(dogs_path))
        self.cats_list = sorted(os.listdir(cats_path))
        self.dogs_len = len(self.dogs_list)
        
    def __len__(self):
        return len(self.dogs_list) + len(self.cats_list)
    
    def getImageIndex(self, index) -> (str, int):
        if index < self.dogs_len:
            return os.path.join(self.dogs_path, self.dogs_list[index]), 0
        return os.path.join(self.cats_path, self.cats_list[index - self.dogs_len]), 1
    
    def __getitem__(self, index):
        id_class = None
        img = None
        sub_index = 0
        while img == None:
            path, id_class = self.getImageIndex(index + sub_index)
            sub_index+=1
            img = cv.imread(path, cv.IMREAD_COLOR)
            
            if img is not None:
                break
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = img.astype(np.float32)
        img = img/255.0
        img = cv.resize(img, (128, 128), interpolation=cv.INTER_AREA)
        img = img.transpose((2, 0, 1))
        t_img = torch.from_numpy(img)
        t_id = torch.tensor(id_class)
        
        return {'img': t_img, 'label': t_id}

In [3]:
dogs_path = './Dataset/PetImages/Dog'
cats_path = './Dataset/PetImages/Cat'
train_dataset = Dataset(dogs_path,cats_path)

### Creating DataLoaders

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True,drop_last=True)

In [5]:
model = tv.models.resnet34(num_classes=2)

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
# count_parameters(model)

In [8]:
# i = 0
# for sample in train_loader:
#     i+=1
#     img = sample['img']
#     label = sample['label']
#     print("i = ", i)
#     if i > 5: break

### Creating Loss function and Optimizer

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, betas=(0.9, 0.999))

In [10]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach(), dim=1).numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()

In [ ]:
epochs = 10

for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample['img'], sample['label']

        optimizer.zero_grad()
        label = F.one_hot(label, 2).float()

 
        pred = model(img)
        loss = loss_fn(pred, label)
        loss.backward()
        

        loss_item = loss.item()
        loss_val += loss_item

        optimizer.step()

        acc_current = accuracy(pred, label)
        acc_val += acc_current

        pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')



  0%|          | 0/390 [00:00<?, ?it/s]